In [1]:
import os
import json
import pandas as pd
import traceback

In [2]:
from langchain.chat_models import ChatOpenAI

In [3]:
from dotenv import load_dotenv # This helps to have access the file .env created to read all value variables we have created

load_dotenv()  # take environment variables

True

In [4]:
KEY=os.getenv("my_openai_key")

In [5]:
llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)

C:\Users\Hp\AppData\Local\Temp\ipykernel_6408\524943024.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  llm = ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)


In [6]:
# Import all necessaries libraries
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [7]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    
}

In [8]:
TEMPLATE="""
Text: {text}
You are an expert MCQ maker. Given the above text, it in your job \
to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your questions like RESPONSE_JSON below and use it as a guide.\
Ensure to make {number} MCQs

### RESPONSE_JSON
{response_json}
"""

In [9]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE

)

In [10]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Hp\AppData\Local\Temp\ipykernel_6408\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [11]:
# Develop an evaluation template for my quizzes. 
TEMPLATE2="""
You are expert english grammarian and writer. Given a Mutiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity\
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilitities.\
Quiz_MCQs:
{quiz}

check from an expert English writer of the above quiz:
"""

In [12]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [13]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [14]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain], input_variables=["text", "number", "subject", "tone", "response_json"], 
                output_variables=["quiz", "review"], verbose=True,)

In [15]:
file_path = r"C:\Users\Hp\Documents\github\mcqstack\data.txt"

In [16]:
file_path

'C:\\Users\\Hp\\Documents\\github\\mcqstack\\data.txt'

In [17]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [18]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [19]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [20]:
NUMBER=5
SUBJECT="machine learning"
Tone="simple"

In [21]:
# https://python.langchain.com/v0.1/docs/modules/model_io/llms/token_usage_tracking/
# How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject": SUBJECT,
            "tone": Tone,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

C:\Users\Hp\AppData\Local\Temp\ipykernel_6408\447738503.py:4: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text: The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied huma

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

You are expert english grammarian and writer. Given a Mutiple Choice Quiz for machine learning students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexityif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the students abilitities.Quiz_MCQs:

{
"1": {
"mcq": "What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?",
"options": {
"a": "Artificial Intelligence",
"b": "Machine Learning",
"c": "Computer Gaming",
"d": "Self-Teaching Computers"
},
"correct": "b"
},
"2": {
"mcq": "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure relevant to machine learning?",
"options": {
"a": "Arthur Samuel",
"b": "Donald Hebb",
"c": "

In [22]:
print(f"Total tokens: {cb.total_tokens}")
print(f"Prompt tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost: {cb.total_cost}")

Total tokens: 1827
Prompt tokens: 1388
Completion Tokens: 439
Total Cost: 0.00296


In [23]:
response

{'text': 'The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]\n\nAlthough the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb\'s model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems

In [24]:
quiz=response.get("quiz")

In [25]:
quiz=quiz.split('\n', 1)[1]


In [26]:
quiz=json.loads(quiz)

In [27]:
quiz

{'1': {'mcq': 'What term was coined in 1959 by Arthur Samuel to describe the field of computer gaming and artificial intelligence?',
  'options': {'a': 'Artificial Intelligence',
   'b': 'Machine Learning',
   'c': 'Computer Gaming',
   'd': 'Self-Teaching Computers'},
  'correct': 'b'},
 '2': {'mcq': "Who published the book 'The Organization of Behavior' in 1949, introducing a theoretical neural structure relevant to machine learning?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Walter Pitts',
   'd': 'Warren McCulloch'},
  'correct': 'b'},
 '3': {'mcq': 'In the 1960s, Raytheon Company developed a learning machine called Cybertron to analyze which types of signals?',
  'options': {'a': 'Radio Signals',
   'b': 'Sonar Signals',
   'c': 'Satellite Signals',
   'd': 'Internet Signals'},
  'correct': 'b'},
 '4': {'mcq': 'According to Tom M. Mitchell, what is the fundamental definition of machine learning algorithms?',
  'options': {'a': 'Improving with experience 

In [28]:
# Create a dataframe
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [f"{option}: {option_value}" for option, option_value in value["options"].items()]
    )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [29]:
quiz=pd.DataFrame(quiz_table_data)

In [30]:
quiz

,MCQ,Choices,Correct
0,What term was coined in 1959 by Arthur Samuel ...,a: Artificial Intelligence | b: Machine Learni...,b
1,Who published the book 'The Organization of Be...,a: Arthur Samuel | b: Donald Hebb | c: Walter ...,b
2,"In the 1960s, Raytheon Company developed a lea...",a: Radio Signals | b: Sonar Signals | c: Satel...,b
3,"According to Tom M. Mitchell, what is the fund...",a: Improving with experience E | b: Classifyin...,d
4,What are the two main objectives of modern-day...,a: To develop models and analyze data | b: To ...,b


In [31]:
quiz.to_csv("machinelearning.csv", index=False)

In [41]:
from datetime import datetime
datetime.now().strftime('%d_%m_%Y_%H_%M_%S')

'30_04_2025_23_25_13'

In [40]:
datetime.now().strftime('%d_%m_%Y_%H_%M_%S')

'30_04_2025_23_25_01'